# Numerics Lab ODE notebook

This ipython notebook is to be used to complete the problem set on the numerics lab.
You will need to execute every cell in this notebook to complete your task.
Some cells will require your input before executing.
They are clearly marked.

Recall that each of these cells (like the one you are reading)
can be modified by double mouse clicking on the cell,
and then executed by pressing `shift+enter`.

## Prerequisites to performing this lab

You should have already read the
[ode_manual.pdf](http://www.usm.uni-muenchen.de/people/puls/lessons/numpraktnew/ode/ode_manual.pdf).
You should also have answered all of the questions in the text up to Chapter&nbsp;4.
This notebook is to be used in lieu of (most of) Chapter&nbsp;4 of the PDF manual.
Important cells in the below notebook have a unique number
that you can use to refer to in your lab write-up.
You can create additional cells to help you organize your work.

Note that this notebook is intended to help you perform the experiments.
It is not a substitute for the written lab report that you must hand in
and that explains your experiments and results in a clear,
comprehensible manner.


# Part&nbsp;2 &mdash; Cosmological application of solutions to ODEs

This part should be done on the second afternoon of the lab work.


In [2]:
#cell #1
# You do not need to modify this cell.
# It is used to import some code dependencies.

%matplotlib qt
import matplotlib.pyplot as plt
import numpy as np
from scipy.integrate import ode
from styles.matplotlib_style import *

## Problem&nbsp;5

#### Exercise&nbsp;15

In cell&nbsp;#32 below, code the function that describes the
evolution of the scale factor of the universe as an ODE.

Neglect the radiation term, since this plays a role
&ldquo;only&rdquo; in the very first epoch(s) of the Universe,
together with inflation, which we will neglect as well,
and which is justified as long as we are not interested
in the details of these phases and have normalized
all quantities to their present values.
Remember that all times are in units of $\tau_H$
if we solve the equation for $\dot a/H_0$.

Note that the function should accept two additional
arguments, $\Omega_M$ and $\Omega_\Lambda$, that describe
the matter density and the cosmological constant.

Since we will only use the RK-integrator with adaptive step-size,
the Jacobian does not need to be defined.


In [67]:
#cell #32

def dydx_friedmann(x, y, omega_m, omega_l, sign=1):
    """
    da / dt  = ....
    Achtung: Time is in units of τ. 
    We assume an expanding universe (plus sign).
    :param x: 
    :param y: 
    :param omega_m: 
    :param omega_l: 
    :return: 
    """
    t = x 
    a = y
    omega_0 = omega_l + omega_m 
    num = 1 - omega_0 + omega_m/a + a**2 * omega_l
    return sign*num**(1/2)
    


In cell&nbsp;#33 below, we have a wrapper routine for the
adaptive-stepsize Runge-Kutta routine that allows passing
an additional parameter list to the function specifying the
system of ODEs.
You can use this to pass $\Omega_M$ and $\Omega_\Lambda$ to
your `dydx` function, allowing you to test different scenarios
without having to rewrite the `dydx` function.
You do not need to modify this cell, but you should understand
how the parameters are passed to the `dydx` function.


In [147]:
#cell #33

def rkdp(dydx: callable,y0:list,a: float,b:float,params: list=None, tol=1e-6):
    """
    Integrate the system of ODEs given by y'(x,y)=dydx(x,y)
    with initial condition y(a)=y0 from a to b using the
    4th/5th-order Dormand-Prince Runge-Kutta method with
    automatic stepsize adjustment controlled by tolerance tol,
    and passing the parameters list to the dydx function.
    """
    print("rkdp:")
    xo = [a]
    nbad = [0]
    nfun = [0]
    def func(x,y,params):
        nfun[0] += 1
        if x<xo[0]:
            nbad[0] += 1
        xo[0] = x
#       print("func:",x,y)
        return dydx(x,y,*params)
    xa = []
    ya = []
    ngood = [0]
    def goodstep(x,y):
        ngood[0] += 1
        xa.append(x)
        ya.append(np.copy(y))
#       print("good:",x,y)
    solver = ode(func)
    solver.set_integrator('dopri5',rtol=tol,atol=1e-15,nsteps=100000,max_step=0.02,verbosity=0)
    solver.set_solout(goodstep)
    solver.set_initial_value(y0,t=a)
    solver.set_f_params(params)
    solver.integrate(b)
    print("\t ",nfun[0],"calls of function")
    print("\t ",ngood[0],"good steps")
    print("\t ",nbad[0],"rejected steps")
    print("\t ",ngood[0]+nbad[0],"total steps")
    return np.array(xa),np.array(ya),nfun[0],ngood[0],nbad[0],ngood[0]+nbad[0]


### Problem&nbsp;5.1

#### Exercise&nbsp;16

Test your program now by means of your results from
Exercise&nbsp;5a/b from the PDF manual.
Plot the results for both scenarios.

As initial value, invert your result from Exercise&nbsp;5a
from the PDF manual (integrated from 0 to $a$) to obtain
an approximate value $a(t_\mathrm{start})$.
In all of the following, we will adopt $t_\mathrm{start}=10^{-10}$
(in units of $\tau_H$).


In [5]:
#cell #34

def analytic_solution_of_a(t_prime):
    """
    Analytic solution of the Friedmann Equation (in Script, equation no.: 3.77)
    
    The equation is 
    
    da/dt / H0 = +sqrt(1/a)
    
    We derived 
    
    a = (3/2 H0 t)**(2/3)
    
    if t = t' τ , where τ = 1/H0
    
    this reduces to 
    
    a = (3/2 t' )**(2/3).
    
    in a flat universe with only a matter component.
    :param t_prime, float:  Time passed since the beginning in units of 1/H0 =: τ.
    :return: 
    """
    return (3/2 * t_prime)**(2/3)

tstart = 1e-10
tend = 1.
astart = analytic_solution_of_a(t_prime=tstart)
print("Our initial value point (t_0, a_0)=(", tstart, astart,")")

# This describes an empty universe:
omega_m_empty = 0
omega_l_empty = 0

# This describes a flat universe with only matter:
omega_m_flatm = 1
omega_l_flatm = 0

t_empty,a_empty,_,_,_,_ = rkdp(dydx_friedmann,astart,tstart,tend,params=(omega_m_empty,omega_l_empty))
t_flatm,a_flatm,nf,ng,nb,nt = rkdp(dydx_friedmann,astart,tstart,tend,params=(omega_m_flatm,omega_l_flatm))

Our initial value point (t_0, a_0)=( 1e-10 2.823108086643088e-07 )
rkdp:
	  332 calls of function
	  56 good steps
	  0 rejected steps
	  56 total steps
rkdp:
	  686 calls of function
	  113 good steps
	  2 rejected steps
	  115 total steps


In [120]:
#cell #35
plot_1 = True
if plot_1:
    plt.suptitle("Notice how the slope is equal to $H_0$")
    plt.title("Evolution of the scale factor according to 1st. Friedmann equation")
    plt.ylabel("$a(t)$")
    plt.plot(t_empty, a_empty, "-", color=blue, label="Empty universe")
    plt.plot(t_flatm, a_flatm, "-", color=red, label="Flat matter universe")
    plt.hlines(1, 0, 1, color="black", ls="--",label="Scale factor $a=1$")
    plt.vlines(2/3, 0, 2, color="black", ls="--",label=r"$t=2/3 \tau$")
    plt.legend()
    plt.xlabel(r"$t$ in units of $\tau$")
    plt.show()
    plt.savefig("Evolution of flat matter and empty universe")


### Problem&nbsp;5.2

## Exercise&nbsp;17

Now, try the solution for a matter-dominated, closed
universe without cosmological constant,  $\Omega_M=3$.
Calculate until $t_\mathrm{max}=3.0$ and plot the result.
Try to explain the observed behavior.
Will the real universe also behave this way?
Why or why not?
If yes, what will the future evolution of the universe be like?
If not, how could you modify the program to reproduce
the &ldquo;real&rdquo; behavior?


In [119]:
#cell #36
omega_m_p52 = 3.
omega_l_p52 = 0.
# From 'closed universe' we infer that Omega_m may be bigger than 1, which is indeed the case here. 

tend = 3.

t_p52,a_p52,nf,ng,nb,nt = rkdp(dydx_friedmann,astart,tstart,tend,params=(omega_m_p52,omega_l_p52), tol=1e-6/1000)


def unit_line(x, t_0):
    return x+t_0


plot_2 = False
if plot_2:
    """plt.suptitle("Notice how the static evolution afters some time is not really physically sensible. \n"
                 "This is because the eigengravitation of matter would have to recollapse and cannot be static. "
                 "\nSo after the static part starts, we should mirror this function in the y-axis, representing the"
                 "negative solution of the equation.")"""
    print("Also: A closed universe should recollapse")
    print("Also: The second Friedmann equation says that deceleration should be negative, i.e. the curvature should be negative!")
    plt.title("Evolution of the scale factor according to 1st. Friedmann equation")
    plt.ylabel("$a(t)$")
    plt.plot(t_p52, a_p52, "-", color=blue, label="Closed matter universe, $\Omega_m=3$", lw=2)
    x_contin = np.linspace(0.2, 0.8, 100)
    plt.plot(x_contin, unit_line(x_contin, 0.5), "--", color="black", label="Unit line")
    plt.legend()
    plt.xlabel(r"$t$ in units of $\tau$")
    plt.savefig("Evolution of high dark energy, closed universe.png")


rkdp:
	  2552 calls of function
	  390 good steps
	  36 rejected steps
	  426 total steps


/var/folders/9y/tnqj0p6s0mjcnrh409r__5l40000gn/T/ipykernel_1715/3410091465.py:18: RuntimeWarning: invalid value encountered in sqrt
  return sign*num**(1/2)


In [8]:
#cell #37

# ...and plot the results. What happens? Is this real or not?


### Problem&nbsp;5.3

#### Exercise&nbsp;18: Solutions for various parameter combinations

After everything runs smoothly, calculate and plot the following models
and briefly explain/comment on your findings.
(Use an **adequate** $t_\mathrm{max}$ &mdash; adapt as needed;
e.g., for expanding universes, first try $t_\mathrm{max}=4$,
but change this if the behavior of the universe can then be seen better.)

a) $\Omega_\mathrm{M}=3.0$, $\Omega_\Lambda=0.1$

b) $\Omega_\mathrm{M}=3.0$, $\Omega_\Lambda=0.2$

c) $\Omega_\mathrm{M}=0.0$, $\Omega_\Lambda=-0.1$

d) $\Omega_\mathrm{M}=1.0$, $\Omega_\Lambda=1.0$

e) $\Omega_\mathrm{M}=1.0$, $\Omega_\Lambda=2.55$

f) $\Omega_\mathrm{M}=1.0$, $\Omega_\Lambda=2.6$


In [160]:
#cell #38

show_sequential_plots_old_version = False
show_sequential_plots_new_version = True



if show_sequential_plots_old_version: 
    plt.figure(figsize=(12, 8))
    # There should be 3 rows and 2 columns
    omega_m_list = [3, 3, 0, 1, 1, 1]
    omega_l_list = [0.1, 0.2, -0.1, 1, 2.55, 2.6]
    t_max_list = [2.5, 7, 5, 5, 4, 2.5]
    counter_index = 1
    plt.suptitle("Evolution of the scale factor according to 1st. Friedmann equation", fontsize=20)
    for omega_m_i, omega_l_i, t_max in zip(omega_m_list, omega_l_list, t_max_list):
        
        plt.subplot(3, 2, counter_index)  # 3 rows, 2 columns, this plot at position counter_index,
        # counted from the top left
        tmax = t_max
        t_p53,a_p53,nf,ng,nb,nt = rkdp(dydx_friedmann,astart,tstart,tmax,params=(omega_m_i,omega_l_i), tol=1e-6)
        if counter_index == 5:
            plt.ylabel("$a(t)$")
            plt.xlabel(r"$t$ in units of $\tau$")
        plt.plot(t_p53, a_p53, "-", color=blue, label="$(\Omega_m, \Omega_{\Lambda})=$"+f"${omega_m_i, omega_l_i}$", lw=2)
        plt.legend()
        
        counter_index += 1
    
    plt.savefig("Playing around with different universes.png")
    
        
def find_static_transition_point(y):
    """Return first point in (x, y) where the gradient of y is zero."""
    grad = np.gradient(y.flatten())
    index_array = np.where(np.isclose(grad, 0))
    return index_array


if show_sequential_plots_new_version: 
    # There should be 3 rows and 2 columns
    omega_m_list = [3, 3, 0, 1, 1, 1]
    omega_l_list = [0.1, 0.2, -0.1, 1, 2.55, 2.6]
    t_max_list = [10, 8, 10, 3, 3, 10]
    colors = ["red", "green", "blue", "violet", "orange", "brown"]
    plt.title("Evolution of the scale factor according to 1st. Friedmann equation", fontsize=20)
    plt.ylabel("$a(t)$")
    plt.xlabel(r"$t$ in units of $\tau$")
    for omega_m_i, omega_l_i, t_max, col in zip(omega_m_list, omega_l_list, t_max_list, colors):
        
        tmax = t_max
    
        time_plus, scale_factor_plus, nf,ng,nb,nt = rkdp(dydx_friedmann,astart,tstart,tmax, params=(omega_m_i,omega_l_i, +1), tol=1e-6)
        
        transition_region = find_static_transition_point(scale_factor_plus)
        # Delete region after transition point in positive solution
        scale_factor = np.delete(scale_factor_plus, transition_region)
        time = np.delete(time_plus, transition_region)
        
        # evolve negative solution starting at transition point
        time_minus, scale_factor_minus, _,_,_,_ = rkdp(dydx_friedmann,scale_factor[-1],time[-1],tmax, params=(omega_m_i,omega_l_i, -1), tol=1e-6)
        
        scale_factor = np.append(scale_factor, scale_factor_minus)
        time = np.append(time, time_minus)
        
        plt.plot(time, scale_factor, "-", color=col, label="$(\Omega_m, \Omega_{\Lambda})=$"+f"${omega_m_i, omega_l_i}$", lw=2)
            
    plt.xlim(0, 11)
    plt.ylim(0, 3.4)
    plt.legend()
    plt.show()
    plt.savefig("Playing around with different universes.png")


rkdp:


/var/folders/9y/tnqj0p6s0mjcnrh409r__5l40000gn/T/ipykernel_1715/3410091465.py:18: RuntimeWarning: invalid value encountered in sqrt
  return sign*num**(1/2)


	  EXIT OF DOPRI5 AT X=        0.2135E+01
  MORE THAN NMAX =      100000 STEPS ARE NEEDED
 600008 calls of function
	  58369 good steps
	  41633 rejected steps
	  100002 total steps
rkdp:
	  1244 calls of function
	  170 good steps
	  38 rejected steps
	  208 total steps
rkdp:
	  2798 calls of function
	  465 good steps
	  2 rejected steps
	  467 total steps
 EXIT OF DOPRI5 AT X=        0.4271E+01
  STEP SIZE T0O SMALL, H=   7.7210412287906631E-015
rkdp:
	  8 calls of function
	  2 good steps
	  1 rejected steps
	  3 total steps
rkdp:
	  3506 calls of function
	  554 good steps
	  31 rejected steps
	  585 total steps
rkdp:
	  1514 calls of function
	  253 good steps
	  1 rejected steps
	  254 total steps
rkdp:
	  1286 calls of function
	  213 good steps
	  2 rejected steps
	  215 total steps
rkdp:
	  8 calls of function
	  2 good steps
	  1 rejected steps
	  3 total steps
rkdp:
	  1286 calls of function
	  213 good steps
	  2 rejected steps
	  215 total steps
rkdp:
	  8 calls of functi

In [25]:
#cell #39

# ...and plot the results. Comment briefly
# on what distinguishes the different universes.


### Problem&nbsp;5.4

#### Exercise&nbsp;19: Constraints on  $\Omega_M$ and $\Omega_\Lambda$

Using the observationally well-proven fact that our
present Universe is very close to flatness,
use your simulations to obtain the $(\Omega_M,\Omega_\Lambda)$ pair
that is consistent with the present Hubble-parameter and
$t_0=13.7\pm 0.2~\mathrm{Gyr}$ (from the WMAP team).
Analyze roughly the corresponding errors.

The present value of $H_0$ is : $67.4 \pm 0.5$


In [123]:
 #cell #40
# 1/H0 =: τ; t0/τ 
# t' = t0/ tau 

tmax = 3
omega_m_54 = 0.33
omega_l_54 = 1-omega_m_54
t0 = 13.7e9 * 365 * 24 * 3600
H0 = 67.4*1000 / (10**6 * 3.26 * 9.461e+15) # 1/s
tau = 1/H0
t_in_units_of_tau = t0 / tau
t_p54,a_p54,nf,ng,nb,nt = rkdp(dydx_friedmann,astart,tstart,tmax,params=(omega_m_54, omega_l_54), tol=1e-6/1000)

"""
Note:
We furthermore calculate the error on the variable `t_in_units_of_tau`=: t_tilde by Gaussian error propagation: 

    t_tilde = H0 * t0,
    
where from the assumed independence of H0 and t0 it follows that 

    Δ t_tilde = ( (H0 * Δt_0)**2  + (t_0 * ΔH0 )**2 )**(-1/2)
    
where Δt_0 = 0.2 Gyrs and ΔΗ0/H0 = 0.00742
"""

H0_relative_error = 0.00742
H0_absolute_error = H0_relative_error * H0

t0_relative_error = 0.0146
t0_absolute_error = t0_relative_error * t0
print("Absolute error of H0 in 1/s:", H0_absolute_error, "value of H0 in 1/s :", H0)
print("Absolute error of t0 in s:", t0_absolute_error, "value of t0 in s :", t0)

t_tilde_absolute_error = ( (H0 * t0_absolute_error)**2 + (t0 * H0_absolute_error)**2)**(1/2)
print("Absolute error of t_tilde: ", t_tilde_absolute_error)

"""plt.suptitle(r"We have experimentally shown that for $(\Omega_m, \Omega_{\Lambda})=(0.33, 0.67)$ that: $a(\tilde{t})=1.$" + "\n"+
            r"We have furthermore shown that $a(\tilde{t}+\Delta\tilde{t})=1$ for $(\Omega_m, \Omega_{\Lambda})=(0.31, 0.69)$, as"
            r"well as $a(\tilde{t}-\Delta\tilde{t})=1$ for $(\Omega_m, \Omega_{\Lambda})=(0.35, 0.65)$.")"""

import matplotlib.gridspec as gridspec

# Create 2x2 sub plots
gs = gridspec.GridSpec(2, 2, height_ratios=[2,1])

plt.subplot(gs[0, :])

plt.vlines(t_in_units_of_tau, -2, 1.4, ls="--", color="black",label="Age of the universe")
plt.hlines(1, -1, 4, ls="--", color="black",label="Scale factor $a(t)=1$")
"""plt.axhspan(1 - 0.1, 1 + 0.1, alpha=0.5, color='blue', 
            label=r"Resulting upper and lower error band on $a(t)=1$.")"""
plt.title("Evolution of the scale factor according to 1st. Friedmann equation")
plt.ylabel("$a(t)$")
plt.plot(t_p54, a_p54, "-", color=blue, label="Universe parameters: $(\Omega_m, \Omega_{\Lambda})=$"+f"${omega_m_54, 0.67}$", lw=2)
plt.axvspan(t_in_units_of_tau - t_tilde_absolute_error, t_in_units_of_tau + t_tilde_absolute_error, alpha=0.5, color='red', 
            label=r"Upper and lower error band on $t^{\prime}/\tau$", ymin=0, ymax=0.3)
plt.xlabel(r"$t$ in units of $\tau$")
plt.legend()
plt.xlim(0, 3)
plt.ylim(0, 5)

plt.subplot(gs[1, 0])
t_p54_min,a_p54_min,_,_,_,_ = rkdp(dydx_friedmann,astart,tstart,tmax,params=(0.35, 1-0.35), tol=1e-6/1000)
plt.plot(t_p54_min, a_p54_min, "-", color=blue, lw=2)
plt.text(0.8573, 1.447, "$(\Omega_m, \Omega_{\Lambda})=$"+f"${0.35, 0.65}$" )
plt.xlim(0.85, 1.05)
plt.ylim(0.8, 1.6)
plt.axvspan(t_in_units_of_tau - t_tilde_absolute_error, t_in_units_of_tau + t_tilde_absolute_error, alpha=0.5, color='red', 
            label=r"Upper and lower error band on $t^{\prime}/\tau$", ymin=0, ymax=1)
plt.vlines(t_in_units_of_tau, -2, 1.4, ls="--", color="black",label="Age of the universe")
plt.hlines(1, -1, 4, ls="--", color="black",label="Scale factor $a(t)=1$")


plt.subplot(gs[1, 1])
t_p54_max,a_p54_max,_,_,_,_ = rkdp(dydx_friedmann,astart,tstart,tmax,params=(0.31, 1-.31), tol=1e-6/1000)
plt.plot(t_p54_max, a_p54_max, "-", color=blue, lw=2)
plt.xlim(0.85, 1.05)
plt.ylim(0.8, 1.6)
plt.axvspan(t_in_units_of_tau - t_tilde_absolute_error, t_in_units_of_tau + t_tilde_absolute_error, alpha=0.5, color='red', 
            label=r"Upper and lower error band on $t^{\prime}/\tau$", ymin=0, ymax=1)
plt.vlines(t_in_units_of_tau, -2, 1.4, ls="--", color="black",label="Age of the universe")
plt.hlines(1, -1, 4, ls="--", color="black", label="Scale factor $a(t)=1$")

plt.text(0.8573, 1.447, "$(\Omega_m, \Omega_{\Lambda})=$"+f"${0.31, 1-.31}$" )

plt.tight_layout()
plt.savefig("Finding omega l and omega m for our universe with error bars.png")

rkdp:
	  1982 calls of function
	  329 good steps
	  2 rejected steps
	  331 total steps
Absolute error of H0 in 1/s: 1.6214709012069567e-20 value of H0 in 1/s : 2.1852707563436075e-18
Absolute error of t0 in s: 6307830720000000.0 value of t0 in s : 4.320432e+17
Absolute error of t_tilde:  0.015462335511548964
rkdp:
	  1982 calls of function
	  329 good steps
	  2 rejected steps
	  331 total steps
rkdp:
	  1982 calls of function
	  329 good steps
	  2 rejected steps
	  331 total steps


### Problem&nbsp;5.5

#### Exercise&nbsp;20: The $(\Omega_M,\Omega_\Lambda)$ diagram

This problem is implemented as a standalone Python program.
Please download the corresponding file from the lab web page.
